In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

pd.set_option('display.max_columns', None)

### TO-DO:
#### 1) Calculate emissions reductions relative to Reference case for each scenario/year.

In [21]:
state='new mexico'
today_date = 'jul12_2021'

In [22]:
tract_area = pd.read_csv('Demographics/demographics_tract.csv')
tract_area.drop(tract_area.columns.difference(['GEOID','tract_area']), 1, inplace=True)
tract_area.rename(columns={'GEOID':'Census Tract', 'tract_area':'Tract Area (sqmi)'},inplace=True)
tract_area.set_index('Census Tract', inplace=True)

In [41]:
subsector_emissions = pd.read_csv('Evolved/my outputs/colorado_transportation_totalEmissions_bySubsector_withSOx_UPDATED_nov23.csv')
subsector_emissions.drop(columns='Unnamed: 0', inplace=True)

In [42]:
subsector_emissions_2017 = subsector_emissions[subsector_emissions['YEAR']==2017]

In [43]:
subsector_emissions_2017_ref = subsector_emissions_2017[subsector_emissions_2017['SCENARIO']=='REFERENCE']

In [46]:
589.502515/1510.494747

0.39027114537856783

In [44]:
subsector_emissions_2017_ref

,SCENARIO,SUBSECTOR,pollutant,YEAR,emissions (metric tons)
3570,REFERENCE,heavy duty trucks,CO,2017,5367.972880
3604,REFERENCE,heavy duty trucks,NOx,2017,18979.171173
3638,REFERENCE,heavy duty trucks,PM10,2017,885.925939
3672,REFERENCE,heavy duty trucks,PM2.5,2017,589.502515
3706,REFERENCE,heavy duty trucks,VOC,2017,1256.656140
3740,REFERENCE,intercity buses,CO,2017,53.281160
3774,REFERENCE,intercity buses,NOx,2017,131.618063
3808,REFERENCE,intercity buses,PM10,2017,9.576203
3842,REFERENCE,intercity buses,PM2.5,2017,6.917626
3876,REFERENCE,intercity buses,VOC,2017,15.353039


In [45]:
subsector_emissions_2017_ref.groupby('pollutant')['emissions (metric tons)'].sum()

pollutant
CO       253578.235177
NOx       46025.032960
PM10       3368.267914
PM2.5      1510.494747
Sox         300.291374
VOC       22174.560105
Name: emissions (metric tons), dtype: float64

In [7]:
lda_inds = subsector_emissions['Source Type']=='light duty autos'
ldt_inds = subsector_emissions['Source Type']=='light duty trucks'
ldv_subsector_emissions = subsector_emissions[lda_inds | ldt_inds]

KeyError: 'Source Type'

In [8]:
inds2020 = subsector_emissions['Year']==2050
inds2030 = subsector_emissions['Year']==2040
pm25_inds = subsector_emissions['Pollutant Type']=='PM2.5'
central_inds = subsector_emissions['Scenario']=='CENTRAL'
ref_inds = subsector_emissions['Scenario']=='REFERENCE'
subsector_emissions_pm = subsector_emissions[inds2020 | inds2030]
subsector_emissions_pm = subsector_emissions_pm[pm25_inds]
subsector_emissions_pm = subsector_emissions_pm[central_inds | ref_inds]
grouped = subsector_emissions_pm.groupby(['Year','State','Sector','Scenario','Pollutant Category','Pollutant Type'])['Emissions (tonnes / year)'].sum().reset_index()

<ipython-input-8-203ee501374c>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsector_emissions_pm = subsector_emissions_pm[pm25_inds]
<ipython-input-8-203ee501374c>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsector_emissions_pm = subsector_emissions_pm[central_inds | ref_inds]


In [9]:
grouped

,Year,State,Sector,Scenario,Pollutant Category,Pollutant Type,Emissions (tonnes / year)
0,2040,NV,Transportation,CENTRAL,CAP,PM2.5,663.682431
1,2040,NV,Transportation,REFERENCE,CAP,PM2.5,1245.806069
2,2050,NV,Transportation,CENTRAL,CAP,PM2.5,297.857409
3,2050,NV,Transportation,REFERENCE,CAP,PM2.5,1392.426249


In [10]:
yeartotal = grouped.groupby(['Year'])['Emissions (tonnes / year)'].sum().reset_index()

### Aggregate CAPs emissions at tract level and calculate cumulative emission reductions.

In [11]:
tract_emissions = subsector_emissions.groupby(['Year','State','County',
                                               'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                                            'Data source'])['Emissions (tonnes / year)'].sum().reset_index()

tract_emissions['Baseline emissions (tonnes / year)'] = tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[0])

tract_emissions['Reference emissions (tonnes / year)'] = tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Pollutant Category','Pollutant Type','Year',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[3])

baseline_inds = tract_emissions['Year']==2017
tract_emissions_baseline = tract_emissions[baseline_inds]
tract_emissions_baseline_area = tract_emissions_baseline.join(tract_area, on='Census Tract')
tract_emissions_baseline_area['Emissions (Tonnes / Square Mile)'] = tract_emissions_baseline_area['Emissions (tonnes / year)'] / tract_emissions_baseline_area['Tract Area (sqmi)']
tract_emissions_baseline_area['Baseline Emissions Percentile'] = tract_emissions_baseline_area.groupby(['Scenario','Year','Pollutant Type'])['Emissions (Tonnes / Square Mile)'].rank(pct=True)

ptile_inds = tract_emissions_baseline_area['Baseline Emissions Percentile']==1
tract_emissions_baseline_area.loc[ptile_inds,'Baseline Emissions Percentile']=.999

tract_emissions_baseline_area.to_csv('Evolved/my outputs/{}_transportation_tractEmissions_baseline2017_{}.csv'.format(state,today_date))

In [12]:
ldv_tract_emissions = ldv_subsector_emissions.groupby(['Year','State','County',
                                               'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                                            'Data source'])['Emissions (tonnes / year)'].sum().reset_index()

In [13]:
ldv_tract_emissions['Baseline LDV emissions (tonnes / year)'] = ldv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[0])

ldv_tract_emissions['Reference LDV emissions (tonnes / year)'] = ldv_tract_emissions.groupby(['State','County',
                            'Census Tract','Sector','Data Category','Pollutant Category','Pollutant Type','Year',
                            'Data source'])['Emissions (tonnes / year)'].transform(lambda x: x.iloc[3])

ldv_baseline_inds = ldv_tract_emissions['Year']==2017
ldv_tract_emissions_baseline = ldv_tract_emissions[ldv_baseline_inds]
ldv_tract_emissions_baseline_area = ldv_tract_emissions_baseline.join(tract_area, on='Census Tract')
ldv_tract_emissions_baseline_area['Emissions (Tonnes / Square Mile)'] = ldv_tract_emissions_baseline_area['Emissions (tonnes / year)'] / ldv_tract_emissions_baseline_area['Tract Area (sqmi)']
ldv_tract_emissions_baseline_area['Baseline Emissions Percentile'] = ldv_tract_emissions_baseline_area.groupby(['Scenario','Year','Pollutant Type'])['Emissions (Tonnes / Square Mile)'].rank(pct=True)

ldv_ptile_inds = ldv_tract_emissions_baseline_area['Baseline Emissions Percentile']==1
ldv_tract_emissions_baseline_area.loc[ldv_ptile_inds,'Baseline Emissions Percentile']=.999

ldv_tract_emissions_baseline_area.to_csv('Evolved/my outputs/{}_transportation_tractLDVEmissions_baseline2017_{}.csv'.format(state,today_date))

In [14]:
ldv_tract_emissions

,Year,State,County,Census Tract,Sector,Data Category,Scenario,Pollutant Category,Pollutant Type,Data source,Emissions (tonnes / year),Baseline LDV emissions (tonnes / year),Reference LDV emissions (tonnes / year)
0,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,CO,Evolved,95.310369,95.310369,95.457274
1,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,NOx,Evolved,11.448368,11.448368,11.463189
2,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM10,Evolved,1.014967,1.014967,1.015302
3,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM2.5,Evolved,0.313459,0.313459,0.313748
4,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP (precursor),VOC,Evolved,10.031133,10.031133,10.049039
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604450,2050,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,NOx,Evolved,0.293625,16.286257,18.601782
604451,2050,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,PM10,Evolved,1.395468,1.351988,1.727889
604452,2050,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,PM2.5,Evolved,0.188872,0.435370,0.482003
604453,2050,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP (precursor),VOC,Evolved,0.259770,13.453447,17.006862


In [15]:
ldv_tract_emissions_baseline_area

,Year,State,County,Census Tract,Sector,Data Category,Scenario,Pollutant Category,Pollutant Type,Data source,Emissions (tonnes / year),Baseline LDV emissions (tonnes / year),Reference LDV emissions (tonnes / year),Tract Area (sqmi),Emissions (Tonnes / Square Mile),Baseline Emissions Percentile
0,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,CO,Evolved,95.310369,95.310369,95.457274,1.098591,86.756943,0.431918
1,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,NOx,Evolved,11.448368,11.448368,11.463189,1.098591,10.420959,0.431918
2,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM10,Evolved,1.014967,1.014967,1.015302,1.098591,0.923881,0.433382
3,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP,PM2.5,Evolved,0.313459,0.313459,0.313748,1.098591,0.285328,0.431918
4,2017,NV,Carson City,32510000100,Transportation,NP,100 PERCENT RENEWABLES,CAP (precursor),VOC,Evolved,10.031133,10.031133,10.049039,1.098591,9.130910,0.433382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17070,2017,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,CO,Evolved,131.683224,131.683224,131.796516,33.533006,3.926973,0.096633
17071,2017,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,NOx,Evolved,16.286257,16.286257,16.298779,33.533006,0.485678,0.096633
17072,2017,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,PM10,Evolved,1.351988,1.351988,1.352260,33.533006,0.040318,0.095168
17073,2017,NV,White Pine County,32033970300,Transportation,NP,SLOW ELECTRICITY,CAP,PM2.5,Evolved,0.435370,0.435370,0.435608,33.533006,0.012983,0.096633


In [16]:
drop_years = [2017,2018,2019]
drop_year_inds = subsector_emissions['Year'].isin(drop_years)
subsector_emissions = subsector_emissions[~drop_year_inds]

drop_year_inds = tract_emissions['Year'].isin(drop_years)
tract_emissions = tract_emissions[~drop_year_inds]

In [17]:
tract_emissions['Cumulative emissions (metric tons)'] = tract_emissions.groupby(['State','County',
                                                                                 'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].transform('cumsum')

tract_emissions['Percentage emission reduction (annual)'] = tract_emissions.groupby(['State','County',
                                                                                     'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].pct_change()

tract_emissions['Percentage emission reduction (cumulative)'] = tract_emissions.groupby(['State','County',
                                                                                         'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].apply(lambda x: x.div(x.iloc[0]).subtract(1))

tract_emissions['Emission reduction (annual)'] = tract_emissions.groupby(['State','County',
                                                                          'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emissions (tonnes / year)'].diff()

tract_emissions['Emission reduction (cumulative)'] = tract_emissions.groupby(['State','County',
                                                                              'Census Tract','Sector','Data Category','Scenario','Pollutant Category','Pollutant Type',
                            'Data source'])['Emission reduction (annual)'].cumsum()



In [18]:
tract_emissions_area = tract_emissions.join(tract_area, on='Census Tract')
emissions_cols = ['Emissions (tonnes / year)', 'Cumulative emissions (metric tons)',
       'Emission reduction (annual)', 'Emission reduction (cumulative)', 'Baseline emissions (tonnes / year)','Reference emissions (tonnes / year)']

for col in emissions_cols:
    tract_emissions_area[col + '_persqmi'] = tract_emissions_area[col] / tract_emissions_area['Tract Area (sqmi)']

In [19]:
tract_emissions_area['Baseline emissions (per sqmi) percentile'] = tract_emissions_area.groupby(['Scenario','Year','Pollutant Type'])['Baseline emissions (tonnes / year)_persqmi'].rank(pct=True)
tract_emissions_area.to_csv('Evolved/my outputs/{}_transportation_tractEmissions_pctChange_{}.csv'.format(state, today_date), float_format='%.10f')

#### Aggregate medium and heavy-duty truck emissions.

#### Check CO demographic data for emissions pct increase.

In [12]:
ejscreen = pd.read_csv('Demographic/Census Tract EJ Index Colorado.csv')
ejscreen.set_index(['TRACT'],inplace=True)
scen_inds = tract_emissions_area['Scenario']=='CENTRAL'
year_inds = tract_emissions_area['Year']==2030
pol_inds = tract_emissions_area['Pollutant Type']=='NOx'
central_year_pol = tract_emissions_area[scen_inds & year_inds & pol_inds]
central_year_pol_ejscreen = central_year_pol.join(ejscreen,on='Census Tract')
emissions_increase = central_year_pol_ejscreen['Percentage emission reduction (cumulative)'] > 0
over75_minority = central_year_pol_ejscreen['LessHSPtile'] > 0.75

### Aggregate NOx, PM2.5, and CO2 emissions at county level for year 2017.

In [21]:
county_emissions = pd.read_csv('Evolved/my outputs/{}_transportation_tractEmissions_bySubsector_CAPsCO2_aug4.csv'.format(state))
county_emissions.drop(columns='Unnamed: 0', inplace=True)

In [22]:
county_emissions_2020 = county_emissions[county_emissions['Year']==2017]
county_emissions_ref2020 = county_emissions_2020[county_emissions_2020['Scenario']=='REFERENCE']
county_emissions_ref2020 = county_emissions_ref2020.groupby(['Year', 'State', 'County', 
        'Sector', 'Data Category', 'Scenario', 'Pollutant Category', 'Pollutant Type',
        'Data source'])['Emissions (tonnes / year)'].sum().reset_index()
pol_inds = county_emissions_ref2020['Pollutant Type'].isin(['NOx','PM2.5','CO2'])
county_emissions_ref2020 = county_emissions_ref2020[pol_inds]

In [24]:
county_emissions_ref2020.to_csv('Evolved/my outputs/{}_transportation_countyEmissions_year2017_CO2NOxPM25_aug9.csv'.format(state))